# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [1]:
from __future__ import print_function, division
SQL_SOURCE = 'https://htv210.carto.com:443/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

#will need to check the last 2 queries
import datetime as dt
import numpy as np

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

In [2]:
test = '''
SELECT *
FROM htv210.citibike_feb_2015
'''

In [3]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',').head()

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.993934,NaN,447,2015-02-01 01:11:00+00,2015-02-01 01:19:00+00,490,8 Ave & W 33 St,40.751551,93,479,9 Ave & W 45 St,40.760193,-73.991255,15185,Subscriber,1983,2
1,NaN,-73.987216,NaN,277,2015-02-01 10:26:00+00,2015-02-01 10:30:00+00,533,Broadway & W 39 St,40.752996,847,515,W 43 St & 10 Ave,40.760094,-73.994618,16452,Subscriber,1978,1
2,NaN,-73.994564,NaN,526,2015-02-01 11:07:00+00,2015-02-01 11:16:00+00,168,W 18 St & 6 Ave,40.739713,1180,521,8 Ave & W 31 St,40.750450,-73.994811,16741,Subscriber,1984,1
3,NaN,-73.989551,NaN,502,2015-02-01 12:37:00+00,2015-02-01 12:46:00+00,402,Broadway & E 22 St,40.740343,1971,459,W 20 St & 11 Ave,40.746745,-74.007756,16951,Subscriber,1983,1
4,NaN,-73.996674,NaN,943,2015-02-01 12:46:00+00,2015-02-01 13:02:00+00,508,W 46 St & 11 Ave,40.763414,2053,521,8 Ave & W 31 St,40.750450,-73.994811,16897,Subscriber,1966,2


In [4]:
queries = ['''SELECT * FROM htv210.citibike_feb_2015
ORDER BY start_station_id''',
           
'''SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration''',
           
 '''SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration
LIMIT 10''',
           
 '''SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration DESC
LIMIT 10''',
           
 '''SELECT * FROM htv210.citibike_feb_2015
WHERE tripduration<10800
ORDER BY tripduration DESC
LIMIT 10''',
           
 '''SELECT * FROM htv210.citibike_feb_2015
WHERE starttime >= '2015-02-02 00:00'
AND starttime < '2015-02-03 00:00'
''',
           
 '''SELECT * FROM htv210.citibike_feb_2015
WHERE extract(DOW FROM starttime) IN (0,6)''',
 '''SELECT * FROM htv210.citibike_feb_2015
WHERE extract(DOW FROM starttime) NOT IN (0,6)''']

In [5]:
queriesRes = []
for query in queries:
    print (query)
    queriesRes.append(pd.read_csv(StringIO.StringIO(queryCartoDB(query, format='CSV', 
                                               source=SQL_SOURCE))))
    print ("\n")

SELECT * FROM htv210.citibike_feb_2015
ORDER BY start_station_id


SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration


SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration
LIMIT 10


SELECT * FROM htv210.citibike_feb_2015
ORDER BY tripduration DESC
LIMIT 10


SELECT * FROM htv210.citibike_feb_2015
WHERE tripduration<10800
ORDER BY tripduration DESC
LIMIT 10


SELECT * FROM htv210.citibike_feb_2015
WHERE starttime >= '2015-02-02 00:00'
AND starttime < '2015-02-03 00:00'



SELECT * FROM htv210.citibike_feb_2015
WHERE extract(DOW FROM starttime) IN (0,6)


SELECT * FROM htv210.citibike_feb_2015
WHERE extract(DOW FROM starttime) NOT IN (0,6)




# checking if it worked for each query

In [6]:
#the columns used to select in the queries
relevantColumns = ["start_station_id", "tripduration", "starttime"]

# 1: all rows sorted by trip station id

In [8]:
#describe the resulting df, only relevant columns - "count" tells you size of df
i = 0
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,46200.000000,46200.000000
mean,439.865216,675.865823
std,328.568800,938.842345
min,72.000000,60.000000
25%,306.000000,354.000000
50%,417.000000,529.000000
75%,491.000000,796.000000
max,3002.000000,43016.000000


In [9]:
#top and bottom (head and tail) of df will tell if sorting was ok
queriesRes[i][relevantColumns].head()

,start_station_id,tripduration,starttime
0,72,1678,2015-02-03 18:21:00+00
1,72,405,2015-02-06 15:07:00+00
2,72,377,2015-02-05 13:46:00+00
3,72,658,2015-02-05 08:48:00+00
4,72,928,2015-02-06 15:53:00+00


In [10]:
queriesRes[i][relevantColumns].tail()

,start_station_id,tripduration,starttime
46195,3002,286,2015-02-01 22:17:00+00
46196,3002,170,2015-02-03 20:40:00+00
46197,3002,132,2015-02-05 17:37:00+00
46198,3002,243,2015-02-06 07:55:00+00
46199,3002,501,2015-02-07 17:10:00+00


# 2: all rows sorted by trip duration

In [11]:
i = 1

In [12]:
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,46200.000000,46200.000000
mean,439.865216,675.865823
std,328.568800,938.842345
min,72.000000,60.000000
25%,306.000000,354.000000
50%,417.000000,529.000000
75%,491.000000,796.000000
max,3002.000000,43016.000000


In [13]:
queriesRes[i][relevantColumns].head()

,start_station_id,tripduration,starttime
0,474,60,2015-02-02 17:14:00+00
1,393,60,2015-02-07 21:40:00+00
2,471,60,2015-02-03 18:01:00+00
3,445,60,2015-02-06 20:40:00+00
4,2009,60,2015-02-05 09:32:00+00


In [14]:
queriesRes[i][relevantColumns].tail()

,start_station_id,tripduration,starttime
46195,438,37665,2015-02-02 08:29:00+00
46196,449,41908,2015-02-06 23:31:00+00
46197,195,42784,2015-02-02 20:07:00+00
46198,492,43013,2015-02-07 23:23:00+00
46199,263,43016,2015-02-02 22:00:00+00


# 3: 10 rows sorted by trip duration shortest trips

In [15]:
i = 2

In [16]:
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,10.000000,10
mean,399.200000,60
std,113.664027,0
min,248.000000,60
25%,301.250000,60
50%,425.000000,60
75%,502.500000,60
max,523.000000,60


In [17]:
queriesRes[i][relevantColumns]

,start_station_id,tripduration,starttime
0,521,60,2015-02-04 06:54:00+00
1,296,60,2015-02-04 08:41:00+00
2,251,60,2015-02-03 08:07:00+00
3,471,60,2015-02-03 18:01:00+00
4,512,60,2015-02-04 07:52:00+00
5,317,60,2015-02-01 14:34:00+00
6,474,60,2015-02-02 17:14:00+00
7,379,60,2015-02-02 21:39:00+00
8,523,60,2015-02-04 22:38:00+00
9,248,60,2015-02-04 18:22:00+00


# 4: 10 rows sorted by trip duration longest trips (sorted longest to shortest)

In [18]:
i = 3

In [19]:
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,10.000000,10.000000
mean,366.100000,37644.100000
std,146.728358,4648.316695
min,83.000000,31396.000000
25%,271.500000,34017.000000
50%,443.500000,36458.000000
75%,482.250000,42565.000000
max,497.000000,43016.000000


In [20]:
queriesRes[i][relevantColumns]

,start_station_id,tripduration,starttime
0,263,43016,2015-02-02 22:00:00+00
1,492,43013,2015-02-07 23:23:00+00
2,195,42784,2015-02-02 20:07:00+00
3,449,41908,2015-02-06 23:31:00+00
4,438,37665,2015-02-02 08:29:00+00
5,453,35251,2015-02-01 00:52:00+00
6,494,35127,2015-02-01 15:47:00+00
7,83,33647,2015-02-04 19:52:00+00
8,297,32634,2015-02-03 22:00:00+00
9,497,31396,2015-02-03 06:19:00+00


# 5: 10 rows sorted by trip duration, longest duration less than 10800 (min) (sorted longest to shortest)

In [21]:
i = 4

In [22]:
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,10.000000,10.00000
mean,417.200000,9951.70000
std,72.985234,461.23266
min,284.000000,9373.00000
25%,390.000000,9606.75000
50%,420.000000,9872.50000
75%,465.250000,10122.75000
max,512.000000,10757.00000


In [23]:
queriesRes[i][relevantColumns]

,start_station_id,tripduration,starttime
0,284,10757,2015-02-07 12:17:00+00
1,512,10660,2015-02-03 17:42:00+00
2,405,10174,2015-02-07 09:31:00+00
3,386,9969,2015-02-02 18:22:00+00
4,468,9938,2015-02-03 14:10:00+00
5,435,9807,2015-02-04 10:53:00+00
6,457,9711,2015-02-05 11:12:00+00
7,402,9572,2015-02-04 19:09:00+00
8,499,9556,2015-02-02 13:25:00+00
9,324,9373,2015-02-03 19:25:00+00


# 6: all trips between feb 2 and feb 3 2015

In [24]:
i = 5

In [25]:
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,1459.000000,1459.000000
mean,435.982865,920.862234
std,326.792747,2369.695501
min,72.000000,60.000000
25%,305.000000,386.500000
50%,414.000000,623.000000
75%,490.000000,923.500000
max,3002.000000,43016.000000


In [26]:
queriesRes[i]["starttime"].min(), queriesRes[i]["starttime"].max()

('2015-02-02 00:02:00+00', '2015-02-02 23:58:00+00')

# 7: weekends days

In [27]:
i = 6

In [28]:
#checking how many rows have weekends in it
np.array([dt.date.weekday(t) < 5 
          for t in pd.to_datetime(queriesRes[i]["starttime"])]).sum()

0

# 8: week days

In [29]:
i = 7

In [30]:
#checking how many rows have day of the week in it
np.array([dt.date.weekday(t) < 5 
          for t in pd.to_datetime(queriesRes[i]["starttime"])]).sum()

32969

In [31]:
# compare to size of df (count)
queriesRes[i][relevantColumns].describe()

,start_station_id,tripduration
count,32969.000000,32969.000000
mean,443.692014,681.052292
std,329.473824,955.523156
min,72.000000,60.000000
25%,311.000000,363.000000
50%,426.000000,537.000000
75%,493.000000,799.000000
max,3002.000000,43016.000000
